In [19]:
import pandas as pd
import numpy as np 
from itertools import permutations
import math
import random

In [20]:
#generate a tree to obtain a additive distance matrix 

class Vertex:
    def __init__(self, node):
        self.id = node
        self.adjacent = {}

    def __str__(self):
        return str([x.id for x in self.adjacent])

    def add_neighbor(self, neighbor, weight=0):
        self.adjacent[neighbor] = weight

    def get_connections(self):
        return self.adjacent.keys()  

    def get_id(self):
        return self.id

    def get_weight(self, neighbor):
        return self.adjacent[neighbor]

class Tree:
    def __init__(self):
        self.vert_dict = {}
        self.num_vertices = 0

    def __iter__(self):
        return iter(self.vert_dict.values())

    def add_vertex(self, node):
        self.num_vertices = self.num_vertices + 1
        new_vertex = Vertex(node)
        self.vert_dict[node] = new_vertex
        return new_vertex

    def get_vertex(self, n):
        if n in self.vert_dict:
            return self.vert_dict[n]
        else:
            return None

    def add_edge(self, frm, to, cost = 0):
        if frm not in self.vert_dict:
            self.add_vertex(frm)
        if to not in self.vert_dict:
            self.add_vertex(to)

        self.vert_dict[frm].add_neighbor(self.vert_dict[to], cost)
        self.vert_dict[to].add_neighbor(self.vert_dict[frm], cost)

    def get_vertices(self):
        return self.vert_dict.keys()

In [21]:
output = (np.array([random.randrange(1, 10) for _ in range(0, 5)]))

#generate n+1 random edge weights where n = # of nodes + a weight for a connecting branch

t = Tree()

t.add_vertex('a')
t.add_vertex('b')
t.add_vertex('c')
t.add_vertex('d')

t.add_edge('a', 'b', output[0] + output[1])   #dist between a to b
t.add_edge('a', 'c', output[0] + output[4] + output[2]) #distance between a, connecting limb, c
t.add_edge('a', 'd', output[0] + output[4] + output[3]) 
t.add_edge('b', 'c', output[1] + output[4] + output[2])
t.add_edge('b', 'd', output[1] + output[4] + output[3])
t.add_edge('c', 'd', output[2] + output[3])

In [22]:
# get dict of dicts for each node, neighbors, distance to neighbors 
def get_matrix(tree):
    D = {}
    for v in tree:
        vid = v.get_id()
        D[vid] = {vid : 0} # assign distance from itself to 0
        for w in v.get_connections():
            wid = w.get_id()
            D[vid][wid] = int(v.get_weight(w))
    return D
        
get_matrix(t)

{'a': {'a': 0, 'b': 16, 'c': 19, 'd': 17},
 'b': {'a': 16, 'b': 0, 'c': 17, 'd': 15},
 'c': {'a': 19, 'b': 17, 'c': 0, 'd': 14},
 'd': {'a': 17, 'b': 15, 'c': 14, 'd': 0}}

In [698]:
mat= pd.DataFrame.from_dict(get_matrix(t)) #convert to 2d matrix
#visualize as n x n distance matrix         
mat

,a,b,c,d
a,0,8,23,20
b,8,0,17,14
c,23,17,0,13
d,20,14,13,0


In [709]:
#check if the matrix is additive using four point condition
def is_additive(matrix, labels):
    i = labels[0]
    j = labels[1] 
    k = labels[2] 
    l = labels[3] 
    result = ''
    if ((matrix[i][j] + matrix[k][l] <= matrix[i][k] + matrix[j][l]) and 
    (matrix[i][k] + matrix[j][l] == matrix[i][l] + matrix[j][k])):
        result = "Additive"
    else:
        result = "Not additive"
    return result 

labels = ['a','b','c','d']
is_additive(mat, labels)

'Additive'

In [30]:
dm = pd.DataFrame.from_dict(tree)
dm['A']['B'] = 8
dm['A']['C'] = 15
dm['A']['D'] = 9
dm['B']['C'] = 5
dm['B']['D'] = 10
dm['C']['D'] = 11

dm['B']['A'] = 8
dm['C']['A'] = 15
dm['D']['A'] = 9
dm['C']['B'] = 5
dm['D']['B'] = 10
dm['D']['C'] = 11
dm

,A,B,C,D
A,0,8,15,9
B,8,0,5,10
C,15,5,0,11
D,9,10,11,0


## Neigbor join

In [31]:
import copy
clusters = []
for cluster in list(dm):
    clusters.append(cluster)
cluster2idx = { name : idx for idx, name in enumerate(clusters) }
idx2cluster = copy.deepcopy(clusters)
m = copy.deepcopy(dm).to_dict()
Z = np.empty((0,4), float)
result = ""
print(clusters)
#membership = [set([cluster2idx[name]]) for name in clusters]

['A', 'B', 'C', 'D']


In [32]:
M, N = dm.shape
new_cluster_idx = M+1
while len(clusters) > 1:
    print(result)
    print(m)
    idx1 = None
    idx2 = None
    ui = 0
    uj = 0
    Sd = 100000
    for i in clusters:
        for j in clusters:
            if i != j:
                ui_t = sum(m[i][k] for k in clusters)
                uj_t = sum(m[j][k] for k in clusters)
                curr = (M - 2) * m[i][j] - ui_t - uj_t
                if Sd > curr:
                    Sd = curr
                    idx1 = cluster2idx[i]
                    idx2 = cluster2idx[j]
                    ui = ui_t
                    uj = uj_t
    print("ui", ui)
    print("uj", uj)
    print(Sd)
    
    Dij = m[idx2cluster[idx1]][idx2cluster[idx2]]
    new_cluster = "(" + idx2cluster[idx1] +"." + idx2cluster[idx2] + ")"
    result = new_cluster
    m[new_cluster] = {}
    new_cluster_idx += 1
    #membership.append(membership[cluster2idx[idx1]] | membership[cluster2idx[idx2]])
    #Z = np.append(Z, np.array([[idx1, idx2, Sd]]))
    if M - 2 != 0:
        m[idx2cluster[idx1]][new_cluster] = (m[idx2cluster[idx1]][idx2cluster[idx2]] + (1/(M-2)) * (ui - uj))/2
        m[idx2cluster[idx2]][new_cluster] = (m[idx2cluster[idx1]][idx2cluster[idx2]] + (1/(M-2)) * (uj - ui))/2
    
    #if idx2cluster[idx1] in m: del m[idx2cluster[idx1]]
    #if idx2cluster[idx2] in m: del m[idx2cluster[idx2]]
    
    clusters.remove(idx2cluster[idx1])
    clusters.remove(idx2cluster[idx2])
    
    m[new_cluster][new_cluster] = 0
    for c in clusters:
        m[c][new_cluster] = (m[c][idx2cluster[idx1]] + m[c][idx2cluster[idx2]] - Dij)/2
        m[new_cluster][c] = m[c][new_cluster]
        for k in list(m[c].keys()):
            if k == idx2cluster[idx1] or k == idx2cluster[idx2]:
                del m[c][k]
        
    del cluster2idx[idx2cluster[idx1]]
    del cluster2idx[idx2cluster[idx2]]
    if idx2cluster[idx1] in m: del m[idx2cluster[idx1]]
    if idx2cluster[idx2] in m: del m[idx2cluster[idx2]]
    
    clusters.append(new_cluster)
    cluster2idx[new_cluster] = len(idx2cluster)
    idx2cluster.append(new_cluster)
        
    M -= 1
    N -= 1


{'A': {'A': 0, 'B': 8, 'C': 15, 'D': 9}, 'B': {'A': 8, 'B': 0, 'C': 5, 'D': 10}, 'C': {'A': 15, 'B': 5, 'C': 0, 'D': 11}, 'D': {'A': 9, 'B': 10, 'C': 11, 'D': 0}}
ui 32
uj 30
-44
(A.D)
{'B': {'B': 0, 'C': 5, '(A.D)': 4.5}, 'C': {'B': 5, 'C': 0, '(A.D)': 8.5}, '(A.D)': {'(A.D)': 0, 'B': 4.5, 'C': 8.5}}
ui 9.5
uj 13.5
-18.0
(B.C)
{'(A.D)': {'(A.D)': 0, '(B.C)': 4.0}, '(B.C)': {'(B.C)': 0, '(A.D)': 4.0}}
ui 4.0
uj 4.0
-8.0


In [33]:
print(result)

((A.D).(B.C))


In [704]:
##additive phlyogeny construction using degenerate triples

In [132]:
dm['A']['B'] = 4
dm['B']['A'] = 4
dm['A']['C'] = 10
dm['C']['A'] = 10
dm['A']['D'] = 9
dm['D']['A'] = 9
dm['B']['C'] = 8
dm['C']['B'] = 8
dm['B']['D'] = 7
dm['D']['B'] = 7
dm['C']['D'] = 9
dm['D']['C'] = 9


def check_and_find_degenerate(D):
    cols = 'ABCD'
    triplets = permutations(D.index,3)
    for triplet in triplets:
        i,j,k = triplet
        if D[i][j] + D[j][k] == D[i][k]:
            return i,j,k, True
        
    return -1,-1,-1, False
    
def compute_trimming_param(D):
    cols = 'ABCD'
    triplets = permutations(D.index,3)
    min_delta = float('inf')
    best_i,best_j,best_k = -1,-1,-1
    for triplet in triplets:
        i,j,k = triplet
        delta = (D[i][j] + D[j][k] - D[i][k])/2
        if delta > 0 and delta < min_delta:
            min_delta = delta
            best_i,best_j,best_k = i,j,k
            
    return min_delta, best_i,best_j,best_k

def Trim(D, delta):
    D = D - (2*delta)
    for i in D.index:
        D[i][i] = 0
    return D

def additive_phylogeny(D, v):
    if D.shape == (2,2):
        T = {D.index[0] : {D.index[1]: D[D.index[0]][D.index[1]]}, 
             D.index[1] : {D.index[0]: D[D.index[1]][D.index[0]]}}
        return T
    i,j,k,flag = check_and_find_degenerate(D)
    if not flag:
        delta,best_i,best_j,best_k = compute_trimming_param(D)
        i,j,k = best_i,best_j,best_k
        D = Trim(D, delta)
    x = D[i][j]
    D = D.drop([j])
    D = D.drop(j, axis=1)
    T = additive_phylogeny(D, v+1)
    T[v] = {i:x}
    T[i] = {v:x}
    if v + 1 in T:
        del T[v+1][i]
        T[v][v+1] = D[i][k]-x-T[v+1][k]
        T[v+1][v] = D[i][k]-x-T[v+1][k]
    else:
        T[v][k] = D[i][k]-x
        T[k] = {v:D[i][k]-x}
    
    T[v][j] = 0
    T[j] = {v:0}
    for l in T:
        if l in D.index and j in D.index and T[l][v] != D[l][j]:
            print('Matrix not additive')
            return
    for v in T:
        for l in 'ABCD':
            if l in T[v] and type(v) == int:
                T[l][v] =  T[l][v] + (delta)
                T[v][l] =  T[v][l] + (delta)
            
    return T


In [133]:
T = additive_phylogeny(dm, 0)

In [134]:
from ete3 import Tree
tree = Tree()
tree_dict = {}
for node in T:
    if node not in tree_dict:
        tree_dict[node] = tree.add_child(name=node, dist=0)
    curr_node = tree_dict[node]
    for edge in T[node]:
        if edge in tree_dict:
            if curr_node not in tree_dict[edge].get_children():
                curr_node.add_child(child=tree_dict[edge], dist=T[node][edge])
        else:
            tree_dict[edge] = curr_node.add_child(name=edge, dist=T[node][edge])
print(tree.write())
print(tree.get_ascii())
print(tree_dict[1].get_children())
print(tree_dict['A'].get_children())

((,(B:1)1:2)1:0,((D:4,(B:1)1:2)1:5)1:0);

   /A /0 /-B
--|
  |      /-D
   \C /1|
         \0 /-B
[Tree node 'D' (0x11c8ec55), Tree node '0' (-0x7fffffffee371331)]
[Tree node '0' (-0x7fffffffee371331)]
